In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random
import time
import sys
import multiprocessing
from collections import OrderedDict, defaultdict
from itertools import product
from copy import deepcopy, copy
from namegenerator import NameGenerator

import gym
import rubiks

# Debugging and profiling
import cProfile
import ipdb

TODO:
    - Add Bias term
    - Custom weights
    - Drop out connections (set weight to 0)
    - Custom activation per node
    
    - Add function that resets stagnation for all species
    - Network Visualisation
    - Species Visualisation
    - Seed

# Activation functions

In [2]:
def leaky_relu(x):
    return F.leaky_relu(x)

def tanh(x):
    return F.Tanh(x)

def relu(x):
    return F.relu(x)

def sigmoid(x):
    return F.Sigmoid(x)

string_to_activation = {
    'leaky_relu' : leaky_relu,
    'relu' : relu,
    'sigmoid' : sigmoid,
    'tanh' : tanh
}

# Model

In [3]:
class Model(nn.Module):
    def __init__(self,layer_sizes):
        super(Model, self).__init__()
        layers = OrderedDict()
        
        previous_layer_size = layer_sizes[0]
        for idx, current_layer_size in enumerate(layer_sizes[1:]):
            layers[str(idx)] = nn.Linear(previous_layer_size, current_layer_size)
            previous_layer_size = current_layer_size
            
        self.layers = nn.Sequential(layers)
        
    def forward(self, x):
        return self.model(x)

In [4]:
firstname_generator = NameGenerator('names.csv', 2, 12)
new_individual_name = firstname_generator.generate_name()
previous_names = []
surname_generator = NameGenerator('surnames.csv', 2, 12)
new_specie_name = surname_generator.generate_name()
previous_surnames = []

# Genotype

In [5]:
class Genotype(object):
    def __init__(self, 
                 inputs = 144, 
                 outputs = 12, 
                 nonlinearities = ['relu','sigmoid','tanh'],
                 topology = None,
                 feedforward = True,
                 max_depth = None,
                 max_nodes = float('inf'),
                 response_default = 4.924273,
                 initial_weight_stdev = 2.0,
                 bias_as_node = False,
                 p_add_neuron = 0.03, 
                 p_add_connection = 0.3, 
                 p_mutate_weight = 0.8,
                 p_reset_weight = 0.1,
                 p_reenable_connection = 0.01,
                 p_disable_connection = 0.01, 
                 p_reenable_parent = 0.25, 
                 p_mutate_bias = 0.2,
                 p_mutate_response = 0.0,
                 p_mutate_type = 0.2,
                 stdev_mutate_weight = 1.5,
                 stdev_mutate_bias = 0.5,
                 stdev_mutate_response = 0.5,
                 weight_range = (-50.,50.),
                 distance_excess_weight = 1.0, 
                 distance_disjoint_weight = 1.0, 
                 distance_weight = 0.4):
        
        self.name = next(new_individual_name)
        self.specie = None
        
        self.inputs = inputs
        self.outputs = outputs
        self.nonlinearities = nonlinearities
        self.feedforward = feedforward
        self.bias_as_node = bias_as_node
        
        self.max_depth = max_depth
        self.max_nodes = max_nodes
        
        self.response_default = response_default
        self.initial_weight_stdev = initial_weight_stdev
        self.stdev_mutate_weight = stdev_mutate_weight
        self.stdev_mutate_bias = stdev_mutate_bias
        self.stdev_mutate_response = stdev_mutate_response
        self.weight_range = weight_range
        
        # Mutation Probabilities
        self.p_add_neuron = p_add_neuron
        self.p_add_connection = p_add_connection
        self.p_mutate_weight = p_mutate_weight
        self.p_reset_weight = p_reset_weight
        self.p_reenable_connection = p_reenable_connection
        self.p_disable_connection = p_disable_connection
        self.p_reenable_parent = p_reenable_parent
        self.p_mutate_bias = p_mutate_bias
        self.p_mutate_response = p_mutate_response
        self.p_mutate_type = p_mutate_type
        
        # Distance weights
        self.distance_excess_weight = distance_excess_weight
        self.distance_disjoint_weight = distance_disjoint_weight
        self.distance_weight = distance_weight
        
        # Tuples of: id, non_linearity, bias, layer, ff_order, response
        self.neuron_genes = []
        # Tuples of: innovation number, input, output, weight, enabled
        self.connection_genes = {}
        # Hyperparameter genes
        self.hyperparameter_genes = []
        
        self.input_ids = []
        self.output_ids = []
        
        self._initialise_topology(topology)
    
    def change_specie(self,specie):
        self.specie = specie
        
    def _initialise_topology(self, topology):
#         if self.bias_as_node:
#             self.inputs += 1
        
        max_layer = 2048 if (self.max_depth is None) else (self.max_depth - 1)
        
        if topology is None:
            # Initialise inputs
            for i in range(self.inputs):
                self.neuron_genes.append([i, random.choice(self.nonlinearities),1.0,0, i * 2048, self.response_default])
                self.input_ids.append(i)
            # Initialise outputs
            for i in range(self.outputs):
                self.neuron_genes.append([(self.inputs + i), random.choice(self.nonlinearities),1.0,max_layer, (self.inputs + i) * 2048, self.response_default])
                self.output_ids.append((self.inputs + i))
            # Initialise connections
            innovation_number = 0
            for i in range(self.inputs):
                for j in range(self.inputs,self.inputs + self.outputs):
                    weight = self._initialise_weight(self.inputs,self.outputs)
                    self.connection_genes[(i,j)] = [innovation_number, i, j, weight ,True]
                    innovation_number += 1
        else:
            raise NotImplementedError

                
    def _initialise_weight(self, input_neurons, output_neurons):
        weight = np.random.rand()*np.sqrt(1/(input_neurons + output_neurons))
        return weight
        
    def recombinate(self, other):
        child = deepcopy(self)
        child.neuron_genes = []
        child.connection_genes = {}
        
        max_neurons = max(len(self.neuron_genes), len(other.neuron_genes))
        min_neurons = min(len(self.neuron_genes), len(other.neuron_genes))
        
        for i in range(max_neurons):
            neuron_gene = None
            if i < min_neurons:
                neuron_gene = random.choice((self.neuron_genes[i], other.neuron_genes[i]))
            else:
                try:
                    neuron_gene = self.neuron_genes[i]
                except IndexError:
                    neuron_gene = other.neuron_genes[i]
            child.neuron_genes.append(deepcopy(neuron_gene))
            
        self_connections = dict(((c[0], c) for c in self.connection_genes.values()))
        other_connections = dict(((c[0], c) for c in other.connection_genes.values()))
        max_innovation_number = max(list(self_connections.keys()) + list(other_connections.keys()))
        
        for i in range(max_innovation_number + 1):
            connection_gene = None
            if i in self_connections and i in other_connections:
                connection_gene = random.choice((self_connections[i],other_connections[i]))
                enabled = self_connections[i][4] and other_connections[i][4]
            else:
                if i in self_connections:
                    connection_gene = self_connections[i]
                    enabled = connection_gene[4]
                elif i in other_connections:
                    connection_gene = other_connections[i]
                    enabled = connection_gene[4]
            if connection_gene is not None:
                child.connection_genes[(connection_gene[1],connection_gene[2])] = deepcopy(connection_gene)
                child.connection_genes[(connection_gene[1],connection_gene[2])][4] = enabled or np.random.rand() < self.p_reenable_parent

            def is_feedforward(item):
                ((fr, to), cg) = item
                return child.neuron_genes[fr][0] < child.neuron_genes[to][0]

            if self.feedforward:
                child.connection_genes = dict(filter(is_feedforward, child.connection_genes.items()))
            
        return child
        
    def mutate(self, innovations = {}, global_innovation_number = 0):
        maximum_innovation_number = max(global_innovation_number, max(cg[0] for cg in self.connection_genes.values()))
        # TODO: move to separate functions
        if len(self.neuron_genes) < self.max_nodes and np.random.rand() < self.p_add_neuron:
            possible_to_split = self.connection_genes.keys()
            
            if self.max_depth is not None:
                possible_to_split = [(fr, to) for (fr, to) in possible_to_split if self.neuron_genes[fr][4] + 1 < self.neuron_genes[to][4]]
            
            if possible_to_split:

                # Choose connection to split
                split_neuron = self.connection_genes[random.choice(list(self.connection_genes.keys()))]
                # Disable old connection
                split_neuron[4] = False

                input_neuron, output_neuron, weight = split_neuron[1:4]
                fforder = (self.neuron_genes[input_neuron][0] + self.neuron_genes[input_neuron][0]) * 0.5
                nonlinearity = random.choice(self.nonlinearities)
                layer = self.neuron_genes[input_neuron][3] + 1
                
                new_id = len(self.neuron_genes)

                neuron = [new_id, nonlinearity, 1.0, layer, fforder, self.response_default]

                self.neuron_genes.append(neuron)
                
                if (input_neuron, new_id) in innovations:
                    innovation_number = innovations[(input_neuron,new_id)]
                else:
                    maximum_innovation_number += 1
                    innovation_number = innovations[(input_neuron,new_id)] = maximum_innovation_number
                    
                # 1.0 to initialise_weight?
                self.connection_genes[(input_neuron, new_id)] = [innovation_number, input_neuron, new_id, 1.0, True]
                
                if (new_id, output_neuron) in innovations:
                    innovation_number = innovations[(new_id, output_neuron)]
                else:
                    maximum_innovation_number += 1
                    innovation_number = innovations[(new_id, output_neuron)] = maximum_innovation_number
                    
                self.connection_genes[(new_id, output_neuron)] = [innovation_number, new_id, output_neuron, weight, True]
                
                
        elif np.random.rand() < self.p_add_connection:
            potential_connections = product(range(len(self.neuron_genes)),range(self.inputs, len(self.neuron_genes)))
            potential_connections = (connection for connection in potential_connections if connection not in self.connection_genes)
            
            if self.feedforward:
                potential_connections = ((f, t) for (f, t) in potential_connections if self.neuron_genes[f][4] < self.neuron_genes[t][4])
            
            potential_connections = list(potential_connections)
            if potential_connections:
                (fr, to) = random.choice(potential_connections)
                if (fr, to) in innovations:
                    innovation = innovations[(fr, to)]
                else:
                    maximum_innovation_number += 1
                    innovation = innovations[(fr, to)] = maximum_innovation_number
                # get number of neurons in layers of fr and to
                connection_gene = [innovation, fr, to, self._initialise_weight(2,2), True]
                self.connection_genes[(fr, to)] = connection_gene
        else:
            for cg in self.connection_genes.values():
                if np.random.rand() < self.p_mutate_weight:
                    cg[3] += np.random.normal(0.0, self.stdev_mutate_weight)
                    cg[3] = np.clip(cg[3], self.weight_range[0], self.weight_range[1])
                    # clipping?
                if np.random.rand() < self.p_reset_weight:
                    cg[3] = np.random.normal(0.0,self.stdev_mutate_weight)
                    
                # bigger chance to disable in this way
                if np.random.rand() < self.p_reenable_connection:
                    cg[4] = True
                    
                if np.random.rand() < self.p_disable_connection:
                    cg[4] = False
                    
            for neuron_gene in self.neuron_genes[self.inputs:]:
                if np.random.rand() < self.p_mutate_bias:
                    neuron_gene[2] += np.random.normal(0.0, 1)

                    neuron_gene[2] = np.clip(neuron_gene[2], self.weight_range[0], self.weight_range[1])
                
                if np.random.rand() < self.p_mutate_type:
                    neuron_gene[1] = random.choice(self.nonlinearities)
                    
                if np.random.rand() < self.p_mutate_response:
                    neuron_gene[5] += np.random.normal(0.0, self.stdev_mutate_response)
                    
        return self
        
    def distance(self, other):
        self_connections = dict(((c[0], c) for c in self.connection_genes.values()))
        other_connections = dict(((c[0], c) for c in other.connection_genes.values()))

        all_innovations = list(self_connections.keys()) + list(other_connections.keys())

        minimum_innovation = min(all_innovations)
        
        e = 0
        d = 0
        w = 0.0
        m = 0
        
        for innovation_key in all_innovations:
            if innovation_key in self_connections and innovation_key in other_connections:
                w += np.abs(self_connections[innovation_key][3] - other_connections[innovation_key][3])
                m += 1
            elif innovation_key in self_connections or innovation_key in other_connections:
                # Disjoint genes
                if innovation_key < minimum_innovation:
                    d += 1
                # Excess genes
                else:
                    e += 1
                    
        # Average weight differences of matching genes
        w = (w/m) if m>0 else w
        
        return (self.distance_excess_weight * e +
               self.distance_disjoint_weight * d +
               self.distance_weight * w)
    
#     def get_network_data(self):
#         """ Returns a tuple of (connection_matrix, node_types) 
#             that is reordered by the "feed-forward order" of the network,
#             Such that if feedforward was set to true, the matrix will be
#             lower-triangular.
#             The node bias is inserted as "node 0", the leftmost column
#             of the matrix.
#         """
        
#         # Assemble connectivity matrix
#         cm = np.zeros((len(self.neuron_genes), len(self.neuron_genes)))
#         cm.fill(np.nan)
        
#         for (_, fr, to, weight, enabled) in self.connection_genes.values():
#             if enabled:
#                 cm[to, fr] = weight
        
#         # Reorder the nodes/connections
# #         ff, node_types, bias, response, layer = zip(*self.neuron_genes)
#         ff, node_types, bias, layer = zip(*self.neuron_genes)
#         order = [i for _,i in sorted(zip(ff, range(len(ff))))]
#         cm = cm[:,order][order,:]
#         node_types = np.array(node_types)[order]
#         bias = np.array(bias)[order]
# #         response = np.array(response)[order]
#         layers = np.array(layer)[order]

# #         # Then, we multiply all the incoming connection weights by the response
#         cm *= np.atleast_2d(4.9).T
#         # Finally, add the bias as incoming weights from node-0
#         if not self.bias_as_node:
#             cm = np.hstack( (np.atleast_2d(bias).T, cm) )
#             cm = np.insert(cm, 0, 0.0, axis=0)
#             # TODO: this is a bit ugly, we duplicate the first node type for 
#             # bias node. It shouldn't matter though since the bias is used as an input.
#             node_types = [node_types[0]] + list(node_types)

#         if self.feedforward and np.triu(np.nan_to_num(cm)).any():
#             import pprint
#             pprint.pprint(self.neuron_genes)
#             pprint.pprint(self.connection_genes)
#             print(ff)
#             print(order)
#             print(np.sign(cm))
#             raise Exception("Network is not feedforward.")
        
#         return cm, node_types

# Species

In [6]:
class Species(object):
    def __init__(self, initial_member):
        self.name = next(new_specie_name)
        self.members = [initial_member]
        self.representative = initial_member
        self.offspring = 0
        self.age = 0
        self.average_fitness = 0.
        self.max_fitness = 0.
        self.max_fitness_previous = 0.0
        self.stagnation = 0
        self.has_best = False

# Population

In [7]:
def evaluate_individual(item):
    (individual, evaluator) = item
    if callable(evaluator):
        individual.stats = evaluator(individual)
    elif hasattr(evaluator, 'evaluate'):
        individual.stats = evaluator.evaluate(individual)
    else:
        raise Exception("Evaluator must be a callable or object" \
                    "with a callable attribute 'evaluate'.")
    return individual

class Population(object):
    def __init__(self, genome_factory,
                population_size = 100,
                elitism = True,
                stop_when_solved = False,
                tournament_selection_k = 3,
                verbose = True,
                max_cores = 1,
                compatibility_threshold = 3.0,
                compatibility_threshold_delta = 0.4,
                target_species = 12,
                minimum_elitism_size = 5,
                young_age = 10,
                young_multiplier = 1.2,
                old_age = 30,
                old_multiplier = 0.2,
                stagnation_age = 15,
                reset_innovations = False,
                survival = 0.2):
        
        self.genome_factory = genome_factory
        self.population_size = population_size
        self.elitism = elitism
        self.stop_when_solved = stop_when_solved
        self.tournament_selection_k = tournament_selection_k
        self.verbose = verbose
        self.max_cores = max_cores
        
        cpus = multiprocessing.cpu_count()
        use_cores = min(self.max_cores, cpus-1)
        if use_cores > 1:
            self.pool = multiprocessing.Pool(processes=use_cores, maxtasksperchild=5)
        else:
            self.pool = None
        
        self.compatibility_threshold = compatibility_threshold
        self.compatibility_threshold_delta = compatibility_threshold_delta
        
        self.target_species = target_species
        self.minimum_elitism_size = minimum_elitism_size
        
        self.young_age = young_age
        self.young_multiplier = young_multiplier
        self.old_age = old_age
        self.old_multiplier = old_multiplier
        
        self.stagnation_age = stagnation_age
        
        self.reset_innovations = reset_innovations
        self.survival = survival
        
    def _evaluate_all(self, population, evaluator):
        to_eval = [(individual, evaluator) for individual in population]
        if self.pool is not None:
            population = list(self.pool.map(evaluate_individual, to_eval))
        else:
            population = list(map(evaluate_individual, to_eval))
        
        return population
        
    def _reset(self):
        self.champions = []
        self.generation = 0
        self.solved_at = None
        self.stats = defaultdict(list)
        self.species = []
        self.global_innovation_number = 0
        self.innovations = {}
        self.current_compatibility_threshold = self.compatibility_threshold
        
    def _find_best(self, population, solution = None):
        self.champions.append(max(population, key=lambda individual: individual.stats['fitness']))
        
        if solution is not None:
            if isinstance(solution, (int, float)):
                solved = (self.champions[-1].stats['fitness'] >= solution)
            elif callable(solution):
                solved = solution(self.champions[-1])
            elif hasattr(solution, 'solve'):
                solved = solution.solve(self.champions[-1])
                
            if solved and self.solved_at is None:
                self.solved_at = self.generation + 1
            
    @property
    def population(self):
        for specie in self.species:
            for member in specie.members:
                yield member
    
    def _evolve(self, evaluator, solution=None):
        population = list(self.population)
        
        while len(population) < self.population_size:
            individual = self.genome_factory()
            population.append(individual)        
            
        population = self._evaluate_all(population, evaluator)
        
        # Speciation
        for specie in self.species:
            # Choose random specie representative for distance comparison
            specie.representative = random.choice(specie.members)
            specie.name = specie.representative.specie
            specie.members = []
            specie.age += 1
            
        # Add each individual to a species
        for individual in population:
            found = False
            for specie in self.species:
                if individual.distance(specie.representative) <= self.current_compatibility_threshold:
                    specie.members.append(individual)
                    individual.change_specie(specie.name)
                    found = True
                    break
            if not found:
                s = Species(individual)
                individual.change_specie(s.name)
                self.species.append(s)
        
        # Remove empty species
        self.species = list(filter(lambda s: len(s.members) > 0, self.species))
        
        # Adjust compatibility threshold
        if len(self.species) < self.target_species:
            self.current_compatibility_threshold -= self.compatibility_threshold_delta
        elif len(self.species) > self.target_species:
            self.current_compatibility_threshold += self.compatibility_threshold_delta
        
        # Find champion and check for solution
        self._find_best(population, solution)
        
        # Recombination
        
        for specie in self.species:
            specie.max_fitness_previous = specie.max_fitness
            specie.average_fitness = np.mean([individual.stats['fitness'] for individual in specie.members])
            specie.max_fitness = np.max([individual.stats['fitness'] for individual in specie.members])
            if specie.max_fitness <= specie.max_fitness_previous:
                specie.stagnation += 1
            else:
                specie.stagnation = 0
            specie.has_best = self.champions[-1] in specie.members
        
        # Keep species that have the best or within stagnation age range
        self.species = list(filter(lambda s: s.stagnation < self.stagnation_age or s.has_best, self.species))
        
        average_fitness = np.array([specie.average_fitness for specie in self.species])
        
        # Adjust fitness based on age
        age = np.array([specie.age for specie in self.species])
        for specie in self.species:
            if specie.age < self.young_age:
                specie.average_fitness *= self.young_multiplier
            if specie.age > self.old_age:
                specie.average_fitness *= self.old_multiplier
                
        # Compute offspring size
        total_fitness = sum(specie.average_fitness for specie in self.species)
        for specie in self.species:
            specie.offspring = int(round(self.population_size * specie.average_fitness / total_fitness))
            
        
        
        # Remove species without offspring
        self.species = list(filter(lambda s: s.offspring > 0, self.species))

        for specie in self.species:
            specie.members.sort(key=lambda individual: individual.stats['fitness'], reverse = True)
            keep = max(1, int(round(len(specie.members)*self.survival)))
            pool = specie.members[:keep]
            
            if self.elitism and len(specie.members) > self.minimum_elitism_size:
                specie.members = specie.members[:1]
            else:
                specie.members = []
                
            while len(specie.members) < specie.offspring:
                k = min(len(pool), self.tournament_selection_k)
                p1 = max(random.sample(pool,k), key=lambda individual: individual.stats['fitness'])
                p2 = max(random.sample(pool,k), key=lambda individual: individual.stats['fitness'])
                
                child = p1.recombinate(p2)
                child.mutate(innovations=self.innovations, global_innovation_number = self.global_innovation_number)
                specie.members.append(child)
                
        if self.innovations:
            self.global_innovation_number = max(self.innovations.values())
            
        self._gather_stats(population)
            
    def epoch(self, evaluator, generations, solution=None, reset=True, callback= None):
        if reset:
            self._reset()
            
        for i in range(generations):
            self.time = time.time()
            self._evolve(evaluator, solution)
            self.generation += 1
            
            if self.verbose:
                self._status_report()
                
            if callback is not None:
                callback(self)
            
            if self.solved_at is not None and self.stop_when_solved:
                break
        
        return {'stats': self.stats, 'champions': self.champions}
    
    def _gather_stats(self, population):
        for key in population[0].stats:
            self.stats[key+'_avg'].append(np.mean([individual.stats[key] for individual in population]))
            self.stats[key+'_max'].append(np.max([individual.stats[key] for individual in population]))
            self.stats[key+'_min'].append(np.min([individual.stats[key] for individual in population]))
        self.stats['solved'].append( self.solved_at is not None )
    
    def _status_report(self):
        print("\n****** Running Generation %d ******" % self.generation)
        fitness_list = np.array([i.stats['fitness'] for i in self.population])
        number_neurons = len(self.champions[-1].neuron_genes)
        number_enabled_connections = np.sum([1 for conn in self.champions[-1].connection_genes.values() if conn[4]])
        print("Population's average fitness: %.5f stdev: %.5f" % (np.average(fitness_list), np.std(fitness_list)))
        print("Best individual: %s %s" % (self.champions[-1].name, self.champions[-1].specie))
        print("Best fitness: %.2f - #neurons: %i - #enabled connections: %i" % (self.champions[-1].stats['fitness'],number_neurons,number_enabled_connections))
        print("Population of %i members in %i species:" % (len(list(self.population)), len(self.species)))
        print("Species         age    size    fitness    stag")
        print("============    ===    ====    =======    ====")
        for specie in self.species:
            print("{: >12}    {: >3}    {: >4}    {:.5f}    {: >4}".format(specie.name,specie.age,len(specie.members),specie.max_fitness,specie.stagnation))
        print("Generation time: %d seconds" % (time.time()-self.time))
        print("Solved in generation: %s" % (self.solved_at))

# Network

In [63]:
def dense_from_coo(shape, conns, dtype=torch.float64):
    mat = torch.zeros(shape, dtype=dtype)
    idxs, weights = conns
    if len(idxs) == 0:
        return mat
    rows, cols = np.array(idxs).transpose()
    mat[torch.tensor(rows), torch.tensor(cols)] = torch.tensor(
        weights, dtype=dtype)
    return mat

def required_for_output(inputs, outputs, connections):
    """
    Collect the nodes whose state is required to compute the final network output(s).
    :param inputs: list of the input identifiers
    :param outputs: list of the output node identifiers
    :param connections: list of (input, output) connections in the network.
    NOTE: It is assumed that the input identifier set and the node identifier set are disjoint.
    By convention, the output node ids are always the same as the output index.
    Returns a set of identifiers of required nodes.
    """
    required = set(outputs)
    s = set(outputs)
    while 1:
        # Find nodes not in S whose output is consumed by a node in s.
        t = set(a for (a, b) in connections if b in s and a not in s)

        if not t:
            break

        layer_nodes = set(x for x in t if x not in inputs)
        if not layer_nodes:
            break

        required = required.union(layer_nodes)
        s = s.union(t)

    return list(required)

class NeuralNetwork():
    def __init__(self, n_inputs, n_hidden, n_outputs,
                 input_to_hidden, hidden_to_hidden, output_to_hidden,
                 input_to_output, hidden_to_output, output_to_output,
                 hidden_responses, output_responses,
                 hidden_biases, output_biases,
                 batch_size=1,
                 activation = 'relu',
                 use_current_activs=False,
                 n_internal_steps=1,
                 dtype=torch.float64):

        self.use_current_activs = use_current_activs
        self.activation = string_to_activation[activation]
        self.n_internal_steps = n_internal_steps
        self.dtype = dtype

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_outputs = n_outputs
        
        if n_hidden > 0:
            self.input_to_hidden = dense_from_coo(
                (n_hidden, n_inputs), input_to_hidden, dtype=dtype)
            self.hidden_to_hidden = dense_from_coo(
                (n_hidden, n_hidden), hidden_to_hidden, dtype=dtype)
            self.output_to_hidden = dense_from_coo(
                (n_hidden, n_outputs), output_to_hidden, dtype=dtype)
            self.hidden_to_output = dense_from_coo(
                (n_outputs, n_hidden), hidden_to_output, dtype=dtype)
        self.input_to_output = dense_from_coo(
            (n_outputs, n_inputs), input_to_output, dtype=dtype)
        self.output_to_output = dense_from_coo(
            (n_outputs, n_outputs), output_to_output, dtype=dtype)
        
        if n_hidden > 0:
            self.hidden_responses = torch.tensor(hidden_responses, dtype=dtype)
            self.hidden_biases = torch.tensor(hidden_biases, dtype=dtype)

        self.output_responses = torch.tensor(
            output_responses, dtype=dtype)
        self.output_biases = torch.tensor(output_biases, dtype=dtype)

        self.reset(batch_size)

    def reset(self, batch_size=1):
        if self.n_hidden > 0:
            self.activs = torch.zeros(
                batch_size, self.n_hidden, dtype=self.dtype)
        else:
            self.activs = None
        self.outputs = torch.zeros(
            batch_size, self.n_outputs, dtype=self.dtype)
        
    def activate(self, inputs):
        '''
        inputs: (batch_size, n_inputs)
        returns: (batch_size, n_outputs)
        '''
        with torch.no_grad():
            inputs = torch.tensor(inputs, dtype=self.dtype)
            activs_for_output = self.activs
            if self.n_hidden > 0:
                for _ in range(self.n_internal_steps):
                    self.activs = self.activation(self.hidden_responses * (
                        self.input_to_hidden.mm(inputs.t()).t() +
                        self.hidden_to_hidden.mm(self.activs.t()).t() +
                        self.output_to_hidden.mm(self.outputs.t()).t()) +
                        self.hidden_biases)
                if self.use_current_activs:
                    activs_for_output = self.activs

            output_inputs = (self.input_to_output.mm(inputs.t()).t() +
                             self.output_to_output.mm(self.outputs.t()).t())
            if self.n_hidden > 0:
                output_inputs += self.hidden_to_output.mm(
                    activs_for_output.t()).t()
            self.outputs = self.activation(
                self.output_responses * output_inputs + self.output_biases)
        return self.outputs
    
    @staticmethod
    def create(genome, batch_size = 1, activation = 'relu', use_current_activs = False, n_internal_steps = 1):
        required = required_for_output(genome.input_ids, genome.output_ids, genome.connection_genes)
        
        input_keys = genome.input_ids
        hidden_keys = [k[0] for k in genome.neuron_genes if k[0] not in genome.output_ids and k[0] not in genome.input_ids]
        output_keys = genome.output_ids
        
#         ipdb.set_trace()
        
        hidden_responses = [genome.neuron_genes[k][5] for k in hidden_keys]
        output_responses = [genome.neuron_genes[k][5] for k in output_keys]

        hidden_biases = [genome.neuron_genes[k][2] for k in hidden_keys]
        output_biases = [genome.neuron_genes[k][2] for k in output_keys]
        
        n_inputs = len(input_keys)
        n_hidden = len(hidden_keys)
        n_outputs = len(output_keys)
        
        input_key_to_idx = {k: i for i, k in enumerate(input_keys)}
        hidden_key_to_idx = {k: i for i, k in enumerate(hidden_keys)}
        output_key_to_idx = {k: i for i, k in enumerate(output_keys)}
        
        def key_to_idx(key):
            if key in input_keys:
                return input_key_to_idx[key]
            elif key in hidden_keys:
                return hidden_key_to_idx[key]
            elif key in output_keys:
                return output_key_to_idx[key]
            
        input_to_hidden = ([], [])
        hidden_to_hidden = ([], [])
        output_to_hidden = ([], [])
        input_to_output = ([], [])
        hidden_to_output = ([], [])
        output_to_output = ([], [])
        
        for connection in genome.connection_genes.values():
            if not connection[4]:
                continue
                
            
            
            if connection[2] not in required and connection[1] not in required:
                continue
            
            input_key = connection[1]
            output_key = connection[2]
                
            if input_key in input_keys and output_key in hidden_keys:
                idxs, vals = input_to_hidden
            elif input_key in hidden_keys and output_key in hidden_keys:
                idxs, vals = hidden_to_hidden
            elif input_key in output_keys and output_key in hidden_keys:
                idxs, vals = output_to_hidden
            elif input_key in input_keys and output_key in output_keys:
                idxs, vals = input_to_output
            elif input_key in hidden_keys and output_key in output_keys:
                idxs, vals = hidden_to_output
            elif input_key in output_keys and output_key in output_keys:
                idxs, vals = output_to_output
                
            idxs.append((key_to_idx(connection[2]), key_to_idx(connection[1])))  # to, from
            vals.append(connection[3])
        
        return NeuralNetwork(n_inputs, n_hidden, n_outputs,
                            input_to_hidden, hidden_to_hidden, output_to_hidden,
                            input_to_output, hidden_to_output, output_to_output,
                            hidden_responses, output_responses,
                            hidden_biases, output_biases,
                            batch_size,
                            activation,
                            use_current_activs,
                            n_internal_steps)

In [64]:
genome = Genotype(2,1)
# genome.neuron_genes.append([3, 'relu', 1.0, 1, 1.0, 4.924273])
# genome.neuron_genes.append([4, 'relu', 1.0, 1, 1.0, 4.924273])
# genome.neuron_genes.append([5, 'sigmoid', 1.0, 1, 0.0, 4.924273])
# genome.connection_genes[(0,2)][4] = False
# genome.connection_genes[(1,2)][4] = False
# genome.connection_genes[(1,3)] = [2, 1, 3, 1.0, True]
# genome.connection_genes[(1,4)] = [4, 1, 4, 1.0, True]
# genome.connection_genes[(0,5)] = [6, 0, 5, 1.0, True]

network = NeuralNetwork.create(genome)

# [[0, 'relu', 1.0, 0, 0, 4.924273], [1, 'tanh', 1.0, 0, 2048, 4.924273], [2, 'relu', 0.5156014272433818, 2048, 4096, 4.924273], [3, 'relu', 1.0, 1, 1.0, 4.924273], [4, 'relu', 1.0, 1, 1.0, 4.924273], [5, 'sigmoid', 1.0, 1, 0.0, 4.924273]]
# {(0, 2): [0, 0, 2, 0.3522098927459531, False], (1, 2): [1, 1, 2, 0.08452610897025808, False], (1, 3): [2, 1, 3, 1.0, True], (1, 4): [4, 1, 4, 1.0, True], (0, 5): [6, 0, 5, 1.0, True], (5, 2): [7, 5, 2, 0.3522098927459531, True]}

# Tasks

In [65]:
class XORTask(object):
    
    # Default XOR input/output pairs
    INPUTS  = [(0,0), (0,1), (1,0), (1,1)]
    OUTPUTS = [(-1,), (1,), (1,), (-1,)]
    EPSILON = 1e-100
    
    def __init__(self, do_all=True):
        self.do_all = do_all
        self.INPUTS = np.array(self.INPUTS, dtype=float)
        self.OUTPUTS = np.array(self.OUTPUTS, dtype=float)
    
    def evaluate(self, network, verbose=False):
        if not isinstance(network, NeuralNetwork):
            network = NeuralNetwork.create(network)
        
        pairs = list(zip(self.INPUTS, self.OUTPUTS))
        random.shuffle(pairs)
        if not self.do_all:
            pairs = [random.choice(pairs)]
        rmse = 0.0
        for (i, target) in pairs:
            # Feed with bias
            output = network.activate(np.array([i]))
            # Grab the output
            output = output[-len(target):]
            err = (target - output)
            err[abs(err) < self.EPSILON] = 0;
            err = (err ** 2).mean()
            # Add error
            if verbose:
                print("%r -> %r (%.2f)" % (i, output, err))
            rmse += err 

        score = 1/(1+np.sqrt(rmse / len(pairs)))
        return {'fitness': score}
        
    def solve(self, network):
        return int(self.evaluate(network)['fitness'] > 0.9)

In [66]:
class RubiksTask(object):
    def __init__(self):
        self.env = rubiks.RubiksEnv(2)
    
    def evaluate(self, network, verbose=False):
        if not isinstance(network, NeuralNetwork):
            network = NeuralNetwork.create(network)
        
        fitness = 0.000001
        
        for i in range(100):
            done = False
            tries = 0
            
            max_tries = 1
            state = self.env.reset(1)
            
            while tries < max_tries and not done:
                action_probabilities = network.feed(state)
                action = np.argmax(action_probabilities)
                
                next_state, reward, done, info = self.env.step(int(action))
                
                tries += 1
                state = next_state
            if done:
                fitness += 1.0
                
        fitness = fitness / 100
        
        return {'fitness' : fitness}
        
    def solve(self, network):
        return int(self.evaluate(network)['fitness'] > 0.5)

In [72]:
inputs = 2
outputs = 1
nonlinearities = ['tanh','relu','sigmoid']
topology = None
feedforward = True
max_depth = None
max_nodes = float('inf')
response_default = 4.924273
bias_as_node = False
initial_weight_stdev = 2.0
p_add_neuron = 0.03
p_add_connection = 0.3
p_mutate_weight = 0.8
p_reset_weight = 0.1
p_reenable_connection = 0.01
p_disable_connection = 0.01
p_reenable_parent=0.25
p_mutate_bias = 0.2
p_mutate_response = 0.0
p_mutate_type = 0.2
stdev_mutate_weight = 1.5
stdev_mutate_bias = 0.5
stdev_mutate_response = 0.5
weight_range = (-50.,50.)

distance_excess_weight = 1.0
distance_disjoint_weight = 1.0
distance_weight = 0.4

In [73]:
population_size = 100
elitism = False
stop_when_solved = True 
tournament_selection_k = 3 
verbose = True
max_cores = 1
compatibility_threshold = 3.0
compatibility_threshold_delta = 0.4 
target_species = 12
minimum_elitism_size = 5 
young_age = 10
young_multiplier = 1.2 
old_age = 30
old_multiplier = 0.2 
stagnation_age = 15
reset_innovations = False
survival = 0.2

genome_factory = lambda: Genotype(inputs, outputs, nonlinearities, topology, feedforward,
                                  max_depth, max_nodes, response_default, initial_weight_stdev,
                                  bias_as_node, p_add_neuron, p_add_connection, p_mutate_weight, 
                                  p_reset_weight, p_reenable_connection, p_disable_connection,
                                  p_reenable_parent, p_mutate_bias, p_mutate_response, p_mutate_type,
                                  stdev_mutate_weight, stdev_mutate_bias, stdev_mutate_response,
                                  weight_range, distance_excess_weight, distance_disjoint_weight,
                                  distance_weight)

population = Population(genome_factory, population_size, elitism, stop_when_solved, tournament_selection_k, verbose, max_cores, compatibility_threshold, compatibility_threshold_delta, target_species, minimum_elitism_size, young_age, young_multiplier, old_age, old_multiplier, stagnation_age, reset_innovations, survival)
task = XORTask()
# task = RubiksTask()

# cProfile.run('population.epoch(evaluator = task, generations = 1, solution = task)', 'restats')
# import pstats
# p = pstats.Stats('restats')
# p.strip_dirs().sort_stats('cumtime').print_stats()
result = population.epoch(evaluator = task, generations = 1000, solution = task)


****** Running Generation 1 ******
Population's average fitness: 0.35709 stdev: 0.02011
Best individual: manue samoi
Best fitness: 0.39 - #neurons: 3 - #enabled connections: 2
Population of 100 members in 1 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
       samoi      0     100    0.38520       0
Generation time: 0 seconds
Solved in generation: None

****** Running Generation 2 ******
Population's average fitness: 0.44779 stdev: 0.00283
Best individual: ur samoi
Best fitness: 0.45 - #neurons: 3 - #enabled connections: 1
Population of 100 members in 1 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
       samoi      1     100    0.44949       0
Generation time: 0 seconds
Solved in generation: None

****** Running Generation 3 ******
Population's average fitness: 0.49830 stdev: 0.04181
Best individual: pru af
Best fitness: 0.54 - #neurons: 4 - #enabled connections: 3
Populat


****** Running Generation 13 ******
Population's average fitness: 0.50026 stdev: 0.07099
Best individual: pru vicox
Best fitness: 0.54 - #neurons: 5 - #enabled connections: 4
Population of 101 members in 13 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
       samoi     12       7    0.53028       0
          af     10       7    0.53495       0
      stisch      8       7    0.50200       3
      helink      8       9    0.52369       1
         cag      6       7    0.50000       1
       vicox      4       8    0.53507       0
         nor      2      11    0.52396       2
         nod      2       7    0.42800       2
      katerb      2       9    0.50000       1
    edamiden      2      10    0.50000       2
      sadonc      1       3    0.12455       1
       reyeo      1       5    0.52244       0
    cavenbel      0      11    0.53507       0
Generation time: 0 seconds
Solved in generation: None

****** Running Generati


****** Running Generation 23 ******
Population's average fitness: 0.51665 stdev: 0.01553
Best individual: pru samoi
Best fitness: 0.54 - #neurons: 8 - #enabled connections: 9
Population of 100 members in 13 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
       samoi     22       8    0.53589       2
          af     20       7    0.53541       4
      stisch     18       8    0.53092       0
      helink     18       7    0.50893       3
         cag     16       8    0.53531       1
      katerb     12       8    0.52496       0
    edamiden     12       8    0.50000       1
       manda      8       7    0.52022       1
        toki      6       6    0.50000       5
        smal      2       9    0.50000       2
        bers      2       8    0.50000       2
       yontu      0       6    0.53589       0
      dochor      0      10    0.50000       0
Generation time: 0 seconds
Solved in generation: None

****** Running Generati


****** Running Generation 33 ******
Population's average fitness: 0.51939 stdev: 0.02585
Best individual: pru dochor
Best fitness: 0.54 - #neurons: 12 - #enabled connections: 13
Population of 101 members in 11 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
       samoi     32       2    0.51318      12
          af     30      10    0.53093       2
      stisch     28      10    0.53574       2
      katerb     22       9    0.50407       0
    edamiden     22      10    0.53585       0
       manda     18       7    0.53531       4
        bers     12      10    0.53541       2
      dochor     10       9    0.53588       0
       oftoc      6      13    0.50000       2
      suredl      3       9    0.44916       2
        feba      1      12    0.53574       1
Generation time: 0 seconds
Solved in generation: None

****** Running Generation 34 ******
Population's average fitness: 0.51732 stdev: 0.03142
Best individual: pru doch


****** Running Generation 43 ******
Population's average fitness: 0.52535 stdev: 0.01896
Best individual: pru dochor
Best fitness: 0.54 - #neurons: 13 - #enabled connections: 16
Population of 98 members in 10 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
          af     40       3    0.51022       0
      stisch     38       3    0.50000       5
      katerb     32       3    0.50000       3
    edamiden     32       1    0.40370       5
       manda     28      12    0.52718       0
        bers     22      14    0.53586       9
      dochor     20      14    0.53588      10
      suredl     13      14    0.50000       6
        elma      7      18    0.53586       5
         per      4      16    0.53586       4
Generation time: 0 seconds
Solved in generation: None

****** Running Generation 44 ******
Population's average fitness: 0.51585 stdev: 0.06622
Best individual: pru ber
Best fitness: 0.54 - #neurons: 14 - #enabled con

...


****** Running Generation 244 ******
Population's average fitness: 0.53149 stdev: 0.00929
Best individual: pru sna
Best fitness: 0.54 - #neurons: 43 - #enabled connections: 109
Population of 99 members in 9 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
       tresn     35       2    0.50000       9
        come     24       9    0.53589       3
         hut     23      11    0.52674       0
   drookinge     17      10    0.50950      13
          bo     14      11    0.53590      14
        cosi     14      11    0.53590      14
    bronterl      2      13    0.53590       2
       gobdu      1      16    0.53589       1
         sna      0      16    0.53590       0
Generation time: 0 seconds
Solved in generation: None

****** Running Generation 245 ******
Population's average fitness: 0.53294 stdev: 0.00686
Best individual: pru bo
Best fitness: 0.54 - #neurons: 43 - #enabled connections: 115
Population of 101 members in 11 spe


****** Running Generation 253 ******
Population's average fitness: 0.53474 stdev: 0.00382
Best individual: pru bronterl
Best fitness: 0.54 - #neurons: 42 - #enabled connections: 111
Population of 99 members in 11 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
         hut     32       2    0.53478       6
   drookinge     26       8    0.52186       6
    bronterl     11       9    0.53590      11
         sna      9       9    0.53590       9
        rock      8      11    0.53589       4
         ort      8       9    0.53590       8
      chorda      7      11    0.53590       1
       lampa      7      10    0.53590       7
        ging      7      10    0.53590       7
          wo      5      10    0.53590       5
         mar      4      10    0.53590       4
Generation time: 1 seconds
Solved in generation: None

****** Running Generation 254 ******
Population's average fitness: 0.53156 stdev: 0.01057
Best individual: pru 


****** Running Generation 263 ******
Population's average fitness: 0.53129 stdev: 0.03924
Best individual: pru bronterl
Best fitness: 0.54 - #neurons: 44 - #enabled connections: 118
Population of 99 members in 6 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
   drookinge     36       1    0.14301       0
    bronterl     21      16    0.53590      21
        rock     18      21    0.53589       0
      chorda     17      19    0.53590      11
         mar     14      19    0.53258       1
       fetig      9      23    0.53590       9
Generation time: 1 seconds
Solved in generation: None

****** Running Generation 264 ******
Population's average fitness: 0.53229 stdev: 0.03396
Best individual: pru bronterl
Best fitness: 0.54 - #neurons: 45 - #enabled connections: 120
Population of 99 members in 7 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
   drookinge     37       1    0


****** Running Generation 272 ******
Population's average fitness: 0.53166 stdev: 0.01004
Best individual: pru chorda
Best fitness: 0.54 - #neurons: 46 - #enabled connections: 128
Population of 99 members in 13 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
        rock     27       7    0.53589       9
      chorda     26       6    0.53590      20
         mar     23       7    0.52214       2
   knerlanew      8       8    0.53590       8
 mandelstade      7       9    0.53590       7
     scoffad      6       8    0.53590       6
hakohammedea      6       8    0.53590       6
        coro      6       7    0.53512       4
      yaguia      6       8    0.53586       6
       pezmi      6       7    0.53590       6
        pepf      6       8    0.53590       6
         nes      5       8    0.53215       0
         cau      4       8    0.50000       3
Generation time: 1 seconds
Solved in generation: None

****** Running Gene

KeyboardInterrupt: 

In [ ]:
print(np.random.rand(2))